In [55]:
import random
import numpy as np

def normalize(arr):
    s = sum(arr)

    if s == 0:
        s = 1
        arr[0] = 1
    
    for i, val in enumerate(arr):
        arr[i] = val/s
    return arr


def generate(width, height):
    matrix = []

    for i in range(height):
        matrix.append([])

        for j in range(width):
            matrix[i].append(float(random.randint(0, 1000))/1000)

        matrix[i] = normalize(matrix[i])

        matrix[i] = [round(x, 3) for x in matrix[i]]
        
    return np.matrix(matrix)

generate(4,4)


print normalize([.444,.45, .34, .0])

[0.35980551053484605, 0.3646677471636953, 0.2755267423014587, 0.0]


In [56]:
#input:num characters
#output: num personality matricies, and closeness vectors

def initialize(num):
    out_matricies = []
    
    for i in range(num * 2):
        out_matricies.append(generate(4,4))
    
    closeness_vectors = []
    for x in range(num):
        close_vector = []
        for i in range(num):
            if i == x:
                close_vector.append(float(0))
            else:
                close_vector.append(float(random.randint(0,1000))/1000)
        closeness_vectors.append(normalize(close_vector))
    return out_matricies, closeness_vectors

initialize(2)

([matrix([[ 0.202,  0.375,  0.166,  0.257],
          [ 0.058,  0.164,  0.502,  0.276],
          [ 0.147,  0.271,  0.405,  0.177],
          [ 0.107,  0.302,  0.395,  0.197]]),
  matrix([[ 0.088,  0.32 ,  0.391,  0.201],
          [ 0.148,  0.291,  0.14 ,  0.422],
          [ 0.189,  0.168,  0.471,  0.172],
          [ 0.139,  0.338,  0.032,  0.492]]),
  matrix([[ 0.233,  0.219,  0.197,  0.351],
          [ 0.28 ,  0.367,  0.237,  0.116],
          [ 0.413,  0.112,  0.04 ,  0.436],
          [ 0.157,  0.213,  0.252,  0.379]]),
  matrix([[ 0.388,  0.153,  0.365,  0.094],
          [ 0.354,  0.173,  0.286,  0.187],
          [ 0.142,  0.412,  0.387,  0.059],
          [ 0.303,  0.33 ,  0.107,  0.26 ]])],
 [[0.0, 1.0], [1.0, 0.0]])

In [57]:
def traverse_row(matrix):
    rand  = float(random.randint(0,1000))/1000
    count = 0
    for i, elem in enumerate(matrix):
        if rand > count and rand < count + elem:
            return i
        count += elem
    return len(matrix) - 1

def translate(num):
    if num == 0:
        return "Happy"
    if num == 1:
        return "Sad"
    if num == 2:
        return "Angry"
    if num == 3:
        return "Fear"     
    
def translate_to_num(string):
    if num == "Happy":
        return 0
    if num == "Sad":
        return 1
    if num == "Angry":
        return 2
    if num == "Fear":
        return 3     

In [40]:
#This takes in one character, all the other characters in the frame, and all the matricies for 
#every character

def new_emotion_one_character(character_acting_id,    #id of acting character in frame
                              character_in_frame_ids, #id of other characters in frame
                              char_emotions,          #array of emotions of characters in frame
                              personality_matricies,  #array of personality matricies of characters in frame
                              impact_matricies,       #array of impact matricies of characters in frame
                              socialbility_params,    #array of socialbility of characters in frame
                              closeness_vectors):     #array of closeness vectors of characters in frame
    
    if len(character_in_frame_ids) == 0:
        P0   = personality_matricies[character_acting_id]
        emo0 = char_emotions[character_acting_id]
        
        if random.randint(0,1) == 1:
            position_and_direction = ['right', 'left']
        else:
            position_and_direction = ['left', 'right']
        
        return (traverse_row(P0.A[emo0]),position_and_direction)
    
    else:
        P0   = personality_matricies[character_acting_id]
        emo0 = char_emotions[character_acting_id]
        s0   = socialbility_params[character_acting_id]
        
        if random.randint(0,1) == 1:
            position_and_direction = ['right', 'left']
        else:
            position_and_direction = ['left', 'right']
        
        total_influence = np.matrix([[0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0]])
        
        for elem in character_in_frame_ids:
            M = impact_matricies[elem].A[char_emotions[elem]]
            v = closeness_vectors[character_acting_id][elem]
            M = np.repeat(M[np.newaxis,:], 4, 0)
            total_influence += v*M
    
        transition_matrix = (1-s0) * P0 + s0 *(total_influence)
    
        #return(traverse_row(transition_matrix.A[emo0]), position_and_direction)
        return(traverse_row(transition_matrix.A[emo0]), transition_matrix)
    

In [39]:
matrixes = initialize(3)
list1, list2 = matrixes
p0, im0, p1, im1, p2, im2 = list1
v0, v1, v2 = list2
soc0 = .3
soc1 = .4
soc2 = .5
emo0 = 0
emo1 = 0
emo2 = 0

print "Before:"
print "char0"
print "Personality Matrix 0:"
print p0
print "Impact Matrix 0:"
print im0
print "emotion:", translate(emo0)
print
print "char1"
print "Personality Matrix 1:"
print p1
print "Impact Matrix 1:"
print im1
print "emotion:", translate(emo1)
print
print "char2"
print "Personality Matrix 2:"
print p2
print "Impact Matrix 2:"
print im2
print "emotion:", translate(emo2)

Before:
char0
Personality Matrix 0:
[[ 0.275  0.26   0.253  0.212]
 [ 0.393  0.553  0.001  0.053]
 [ 0.106  0.425  0.432  0.037]
 [ 0.098  0.092  0.388  0.423]]
Impact Matrix 0:
[[ 0.04   0.36   0.3    0.3  ]
 [ 0.67   0.088  0.235  0.006]
 [ 0.243  0.077  0.206  0.475]
 [ 0.107  0.229  0.285  0.379]]
emotion: Happy

char1
Personality Matrix 1:
[[ 0.136  0.088  0.471  0.306]
 [ 0.367  0.185  0.259  0.19 ]
 [ 0.215  0.216  0.335  0.233]
 [ 0.45   0.471  0.023  0.057]]
Impact Matrix 1:
[[ 0.343  0.32   0.113  0.224]
 [ 0.263  0.326  0.287  0.124]
 [ 0.388  0.143  0.398  0.071]
 [ 0.03   0.434  0.161  0.375]]
emotion: Happy

char2
Personality Matrix 2:
[[ 0.039  0.312  0.284  0.366]
 [ 0.073  0.333  0.465  0.129]
 [ 0.37   0.237  0.24   0.153]
 [ 0.136  0.036  0.7    0.128]]
Impact Matrix 2:
[[ 0.038  0.124  0.461  0.377]
 [ 0.066  0.323  0.442  0.17 ]
 [ 0.11   0.299  0.37   0.221]
 [ 0.417  0.189  0.384  0.009]]
emotion: Happy


In [54]:

emo_0, direction_0 = new_emotion_one_character( 0, [1,2], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])
emo_1, direction_1 = new_emotion_one_character( 1, [0,2], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])
emo_2, direction_2 = new_emotion_one_character( 2, [0,1], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])

print "After interaction:"
print "char0 emotion:", translate(emo_0)
print "char1 emotion:", translate(emo_1)
print "char2 emotion:", translate(emo_2)

print direction_0
print direction_1
print direction_2



After interaction:
char0 emotion: Angry
char1 emotion: Angry
char2 emotion: Fear
[[ 0.22834178  0.23490685  0.28751233  0.24923904]
 [ 0.31094178  0.44000685  0.11111233  0.13793904]
 [ 0.11004178  0.35040685  0.41281233  0.12673904]
 [ 0.10444178  0.11730685  0.38201233  0.39693904]]
[[ 0.09749146  0.18399251  0.41133731  0.30777871]
 [ 0.23609146  0.24219251  0.28413731  0.23817871]
 [ 0.14489146  0.26079251  0.32973731  0.26397871]
 [ 0.28589146  0.41379251  0.14253731  0.15837871]]
[[ 0.09383544  0.328827    0.25846624  0.31937131]
 [ 0.11083544  0.339327    0.34896624  0.20087131]
 [ 0.25933544  0.291327    0.23646624  0.21287131]
 [ 0.14233544  0.190827    0.46646624  0.20037131]]
